# Causal prediction with `TIGRAMITE`

TIGRAMITE is a time series analysis python module. It allows to reconstruct graphical models (conditional independence graphs) from discrete or continuously-valued time series based on the PCMCI framework and create high-quality plots of the results.

PCMCI is described here:
J. Runge, P. Nowack, M. Kretschmer, S. Flaxman, D. Sejdinovic, 
Detecting and quantifying causal associations in large nonlinear time series datasets. Sci. Adv. 5, eaau4996 (2019) 
https://advances.sciencemag.org/content/5/11/eaau4996

For further versions of PCMCI (e.g., PCMCI+, LPCMCI, etc.), see the corresponding tutorials.

This tutorial explains how to use PCMCI to obtain optimal predictors. See the following paper for theoretical background:
Runge, Jakob, Reik V. Donner, and Jürgen Kurths. 2015. “Optimal Model-Free Prediction from Multivariate Time Series.” Phys. Rev. E 91 (5): 052909. https://doi.org/10.1103/PhysRevE.91.052909.

Last, the following Nature Communications Perspective paper provides an overview of causal inference methods in general, identifies promising applications, and discusses methodological challenges (exemplified in Earth system sciences): 
https://www.nature.com/articles/s41467-019-10105-3

In [1]:
# Imports
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline     
## use `%matplotlib notebook` for interactive figures
# plt.style.use('ggplot')
import sklearn

import tigramite
from tigramite import data_processing as pp
from tigramite.toymodels import structural_causal_processes as toys
from tigramite import plotting as tp
from tigramite.pcmci import PCMCI

from tigramite.independence_tests.parcorr import ParCorr
from tigramite.independence_tests.gpdc import GPDC
from tigramite.independence_tests.cmiknn import CMIknn
from tigramite.independence_tests.cmisymb import CMIsymb

from tigramite.models import LinearMediation, Prediction

AttributeError: 'RcParams' object has no attribute '_get'

# Prediction

Tigramite also contains a class ``tigramite.models.Prediction`` to perform predictions based on the sklearn models. The ``Prediction`` class includes a wrapper around ``run_pc_stable`` from the ``PCMCI`` class to perform predictor selection. Consider the following data generation process: 

In [ ]:
np.random.seed(42)
T = 200
links_coeffs = {0: [((0, -1), 0.6)],
                1: [((1, -1), 0.6), ((0, -1), 0.8)],
                2: [((2, -1), 0.5), ((1, -1), 0.7)],  # ((0, -1), c)],
                }
N = len(links_coeffs)
data, true_parents = toys.var_process(links_coeffs, T=T)
dataframe = pp.DataFrame(data, var_names = [r'$X^0$', r'$X^1$', r'$X^2$'])

We initialize the ``Prediction`` class with ``cond_ind_model=ParCorr()``. Secondly, we choose ``sklearn.linear_model.LinearRegression()`` here for prediction. Last, we scale the data via ``data_transform``. The class takes care of rescaling the data for prediction. The parameters ``train_indices`` and ``test_indices`` are used to divide the data up into a training set and test set. Here we insert a gap of at least ``tau_max`` (chosen to be 30 below) between train and test indices, because we will use lagged predictors to predict on the test indices. The test set is optional since new data can be supplied later. The training set is used to select predictors and fit the model.

In [ ]:
pred = Prediction(dataframe=dataframe,
        cond_ind_test=ParCorr(),   #CMIknn ParCorr
        prediction_model = sklearn.linear_model.LinearRegression(),
#         prediction_model = sklearn.gaussian_process.GaussianProcessRegressor(),
        # prediction_model = sklearn.neighbors.KNeighborsRegressor(),
    data_transform=sklearn.preprocessing.StandardScaler(),
    train_indices= range(int(0.8*T)),
    test_indices= range(int(0.9*T), T),
    verbosity=1
    )

Now, we estimate causal predictors using ``get_predictors`` for the target variable 2 taking into account a maximum past lag of ``tau_max``. We use ``pc_alpha=None`` which optimizes the parameter based on the Akaike score. Note that the predictors are different for each prediction horizon. For example, at a prediction horizon of ``steps_ahead=1`` we get the causal parents from the model plus some others:

In [ ]:
target = 2
tau_max = 5
predictors = pred.get_predictors(
                  selected_targets=[target],
                  steps_ahead=1,
                  tau_max=tau_max,
                  pc_alpha=None
                  )
graph = np.zeros((N, N, tau_max+1), dtype='bool')
for j in [target]:
    for p in predictors[j]:
        graph[p[0], j, abs(p[1])] = 1

# Plot time series graph
tp.plot_time_series_graph(
    figsize=(6, 3),
#     node_aspect=2.,
    val_matrix=np.ones(graph.shape),
    graph=graph,
    var_names=None,
    link_colorbar_label='',
    ); plt.show()


##
## Step 1: PC1 algorithm with lagged conditions
##

Parameters:
independence test = par_corr
tau_min = 1
tau_max = 5
pc_alpha = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5]
max_conds_dim = None
max_combinations = 1



## Resulting lagged parent (super)sets:

    Variable $X^0$ has 1 link(s):
    [pc_alpha = 0.05]
        ($X^0$ -1): max_pval = 0.00000, min_val =  0.560

    Variable $X^1$ has 3 link(s):
    [pc_alpha = 0.3]
        ($X^0$ -1): max_pval = 0.00000, min_val =  0.685
        ($X^1$ -1): max_pval = 0.00000, min_val =  0.556
        ($X^2$ -1): max_pval = 0.22735, min_val = -0.099

    Variable $X^2$ has 3 link(s):
    [pc_alpha = 0.1]
        ($X^1$ -1): max_pval = 0.00000, min_val =  0.557
        ($X^2$ -1): max_pval = 0.00000, min_val =  0.418
        ($X^1$ -2): max_pval = 0.05793, min_val =  0.157


AttributeError: 'RcParams' object has no attribute '_get'

Note, that ``get_predictors`` is based only on the first step of PCMCI and skips the MCI step since correct false positive rates are not that relevant for prediction and the first step alone is faster. Now, we set ``steps_ahead=2`` and get different predictors:

These predictors now efficiently avoid overfitting in the following model fit. Here one can specify whether multiple target variables should be fit at once (assuming that for all of these predictors have been estimated beforehand).

In [ ]:
pred.fit(target_predictors=predictors, 
                selected_targets=[target],
                    tau_max=tau_max)

Now we are ready to predict the target variable at the test samples (NMAE is the mean absolute error normalized by the std):

In [ ]:
print(target)
predicted = pred.predict(target)
true_data = pred.get_test_array(j=target)[0]

plt.scatter(true_data, predicted)
plt.title(r"NMAE = %.2f" % (np.abs(true_data - predicted).mean()/true_data.std()))
plt.plot(true_data, true_data, 'k-')
plt.xlabel('True test data')
plt.ylabel('Predicted test data')
plt.show()